In [7]:
import labdatatools as ldt
from pathlib import Path
import numpy as np
import subprocess

SUBJECT = 'JC131'
N_PROBES = 1

In [8]:
files = ldt.rclone_list_files(SUBJECT).dropna()
ori_sessions = files[files.datatype == 'orientation'].session
ori_sessions = np.unique(ori_sessions).tolist()
print(len(ori_sessions))

temp = []
for sess in ori_sessions: # ensure that ephys data exists
    #if 'ephys_g0' in files[files.session == sess].datatype.values:
    if np.any(files[files.session == sess].datatype.str.contains('ephys_')):
        ses = files[files.session == sess].iloc[0].session
        temp.append(ses)
ori_sessions_with_ephys = np.array(temp)
print(len(ori_sessions_with_ephys))

91
76


In [9]:
temp, temp2 = [],[]
for sess in ori_sessions_with_ephys:
    ses = files[files.session == sess].iloc[0].session
    if np.any(files[files.session == sess].datatype.str.contains('kilosort2.5')):
        temp.append(ses)
    else:
        temp2.append(ses)
sorted_ori_sessions = np.array(temp)
unsorted_ori_sessions = np.array(temp2)
print(len(sorted_ori_sessions))
print(len(unsorted_ori_sessions))
    

76
0


In [10]:
unsorted_ori_sessions

array([], dtype=float64)

In [11]:
#submit the jobs
submit_cmd = 'labdata submit spks -a {subject} -s {session} --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=3:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR'
submit_cmd = 'labdata submit spks -a {subject} -s {session} -g 1 -n 6 -m 40 --delete-session --overwrite -- -t /scratch'

for sess in unsorted_ori_sessions:
    cmd = submit_cmd.format(session=sess, subject=SUBJECT)
    print(cmd)
    #out = subprocess.check_output(cmd.split(' ')).decode()
    #print(out)